In [1]:
%pip install contractions
%pip install textblob
%pip install nltk
%pip install scikit-learn
%pip install transformers
%pip install emoji
%pip install spacy
!python -m spacy download en_core_web_sm


[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.

[notice] A new release of pip is available: 24.0 -> 25.0
[notice] To update, run: pip3 install --upgrade pip
Note: you 

In [2]:
import pandas as pd
import re
from sklearn.feature_extraction import text
import string
import contractions
import emoji
import spacy
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

Loading the data:

In [3]:
data = pd.read_csv("combined_data.csv")
data.head()

/var/folders/mj/x2186rjd6njg5v_9jd7x61kh0000gn/T/ipykernel_3929/2371751024.py:1: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv("combined_data.csv")


,post_id,comment_id,title,body,subreddit,upvotes,comments,date_time,author,query
0,1hr4hc6,NaN,"Weekly Self-Promotional Mega Thread 49, 01.01....",All the self-promotional posts about your AI p...,ChatGPT,23,118,2025-01-01 14:58:15,pirate_jack_sparrow_,NaN
1,1ggixzy,NaN,"AMA with OpenAI’s Sam Altman, Kevin Weil, Srin...",Consider this AMA our Reddit launch.\n\nAsk us...,ChatGPT,4000,4705,2024-10-31 16:40:38,OpenAI,NaN
2,1id5l47,NaN,OpenAI Pleads That It Can’t Make Money Without...,"# ""It would be impossible to train today’s lea...",ChatGPT,1135,182,2025-01-29 21:45:58,faustoc5,NaN
3,1icyjx6,NaN,Remember When OpenAI Threatened Your Job? A Fr...,NaN,ChatGPT,1222,131,2025-01-29 16:59:46,Dexter01010,NaN
4,1icvvjq,NaN,"""My AI just absolutely roasted me and I'm ques...",NaN,ChatGPT,1362,159,2025-01-29 15:08:12,maruhadapurpurine,NaN


In [4]:
data["text"] = data.apply(lambda row: f"{row['title']} {row['body']}" if pd.isna(row['comment_id']) else row['body'], axis=1)
data["text"].head()

0    Weekly Self-Promotional Mega Thread 49, 01.01....
1    AMA with OpenAI’s Sam Altman, Kevin Weil, Srin...
2    OpenAI Pleads That It Can’t Make Money Without...
3    Remember When OpenAI Threatened Your Job? A Fr...
4    "My AI just absolutely roasted me and I'm ques...
Name: text, dtype: object

Removing blank rows:

In [5]:
# Remove Blank Rows
data = data.dropna(subset=["text"])
data = data.reset_index(drop=True)
data.head()

,post_id,comment_id,title,body,subreddit,upvotes,comments,date_time,author,query,text
0,1hr4hc6,NaN,"Weekly Self-Promotional Mega Thread 49, 01.01....",All the self-promotional posts about your AI p...,ChatGPT,23,118,2025-01-01 14:58:15,pirate_jack_sparrow_,NaN,"Weekly Self-Promotional Mega Thread 49, 01.01...."
1,1ggixzy,NaN,"AMA with OpenAI’s Sam Altman, Kevin Weil, Srin...",Consider this AMA our Reddit launch.\n\nAsk us...,ChatGPT,4000,4705,2024-10-31 16:40:38,OpenAI,NaN,"AMA with OpenAI’s Sam Altman, Kevin Weil, Srin..."
2,1id5l47,NaN,OpenAI Pleads That It Can’t Make Money Without...,"# ""It would be impossible to train today’s lea...",ChatGPT,1135,182,2025-01-29 21:45:58,faustoc5,NaN,OpenAI Pleads That It Can’t Make Money Without...
3,1icyjx6,NaN,Remember When OpenAI Threatened Your Job? A Fr...,NaN,ChatGPT,1222,131,2025-01-29 16:59:46,Dexter01010,NaN,Remember When OpenAI Threatened Your Job? A Fr...
4,1icvvjq,NaN,"""My AI just absolutely roasted me and I'm ques...",NaN,ChatGPT,1362,159,2025-01-29 15:08:12,maruhadapurpurine,NaN,"""My AI just absolutely roasted me and I'm ques..."


Converting texts to lowercase:

In [6]:
# Lowercase
def lower(text):
  return text.lower()

data["Cleaned Text"] = data["text"].apply(lower)
data["Cleaned Text"].head()

0    weekly self-promotional mega thread 49, 01.01....
1    ama with openai’s sam altman, kevin weil, srin...
2    openai pleads that it can’t make money without...
3    remember when openai threatened your job? a fr...
4    "my ai just absolutely roasted me and i'm ques...
Name: Cleaned Text, dtype: object

Removing r/, usernames, new line indicators, and links from texts:

In [7]:
# Remove unnecessary tokens
def remove_links(text):
  return re.sub(r'http[s]?://\S+|www\.\S+', '', text)

def remove_user_mentions(text):
    return re.sub(r'u/\S+', '', text)

data["Cleaned Text"] = data["Cleaned Text"].str.replace('r/', '', regex=False)
data["Cleaned Text"] = data["Cleaned Text"].str.replace("\n\n", ' ', regex=False)
data["Cleaned Text"] = data["Cleaned Text"].apply(remove_links)
data["Cleaned Text"] = data["Cleaned Text"].apply(remove_user_mentions)

data["Cleaned Text"].head()

0    weekly self-promotional mega thread 49, 01.01....
1    ama with openai’s sam altman, kevin weil, srin...
2    openai pleads that it can’t make money without...
3    remember when openai threatened your job? a fr...
4    "my ai just absolutely roasted me and i'm ques...
Name: Cleaned Text, dtype: object

Fixing spelling errors:

In [8]:
# # Fix spelling
# def correct_spelling(text):
#     return str(TextBlob(text).correct())

# data["Cleaned Text"] = data["Cleaned Text"].apply(correct_spelling)
# data["Cleaned Text"].head()

Expanding contractions:

In [9]:
# Expand contractions
def expand_contractions(text):
    return contractions.fix(text)

data["Cleaned Text"] = data["Cleaned Text"].apply(expand_contractions)
data["Cleaned Text"].head()

0    weekly self-promotional mega thread 49, 01.01....
1    ama with openai’s sam altman, kevin weil, srin...
2    openai pleads that it cannot make money withou...
3    remember when openai threatened your job? a fr...
4    "my ai just absolutely roasted me and i am que...
Name: Cleaned Text, dtype: object

Removing stop words:

In [10]:
# Remove stopwords
stopwords = text.ENGLISH_STOP_WORDS

data["Cleaned Text"] = data["Cleaned Text"].apply(
    lambda text: ' '.join([word for word in text.split() if word.lower() not in stopwords])
)
data["Cleaned Text"].head()

0    weekly self-promotional mega thread 49, 01.01....
1    ama openai’s sam altman, kevin weil, srinivas ...
2    openai pleads make money using copyrighted mat...
3    remember openai threatened job? free ai just r...
4    "my ai just absolutely roasted questioning rea...
Name: Cleaned Text, dtype: object

Removing punctuations:

In [11]:
# Remove punctuations
def remove_punctuations(text):
    text = re.sub(r'[-]', ' ', text)
    text = re.sub(r'(\S)[' + re.escape(string.punctuation) + r'](\S)', r'\1 \2', text)
    return text

data["Cleaned Text"] = data["Cleaned Text"].apply(remove_punctuations)

pattern_punctuations = r'[' + string.punctuation + r']'

data["Cleaned Text"] = data["Cleaned Text"].str.replace(pattern_punctuations, '', regex=True)

data["Cleaned Text"].head()

0    weekly self promotional mega thread 49 01 01 2...
1    ama openai’s sam altman kevin weil srinivas na...
2    openai pleads make money using copyrighted mat...
3    remember openai threatened job free ai just re...
4    my ai just absolutely roasted questioning real...
Name: Cleaned Text, dtype: object

Converting emojis to their descriptive names:

In [12]:
def convert_emojis(text):
   return emoji.demojize(text)

data["Cleaned Text"] = data["Cleaned Text"].apply(convert_emojis)
data["Cleaned Text"].head()

0    weekly self promotional mega thread 49 01 01 2...
1    ama openai’s sam altman kevin weil srinivas na...
2    openai pleads make money using copyrighted mat...
3    remember openai threatened job free ai just re...
4    my ai just absolutely roasted questioning real...
Name: Cleaned Text, dtype: object

: 

### TF-IDF Vectorizer 

In [ ]:
nlp = spacy.load("en_core_web_sm")

def custom_tokenizer(text):
    doc = nlp(text)
    tokens = []
    for ent in doc.ents:
        tokens.append(ent.text)

    non_entity_tokens = [token.text for token in doc if not token.ent_type_ and not token.is_punct and not token.is_space]
    tokens.extend(non_entity_tokens)
    return tokens

vectorizer = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    use_idf=False
)

doc_vectors = vectorizer.fit_transform(data["Cleaned Text"])

feature_names = vectorizer.get_feature_names_out()
dense_vec = doc_vectors.todense()
dense_list = dense_vec.tolist()
tfidf_data = pd.DataFrame(dense_list, columns=feature_names)
tfidf_data

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/sklearn/feature_extraction/text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [ ]:
query = ["Discussions about ChatGPT, its performance, user experiences, applications, limitations, ethical concerns, and comparisons with other AI models developed by OpenAI."]

idf_vectorizer = TfidfVectorizer(
    tokenizer=custom_tokenizer,
    use_idf=True,
    # turn off sublinear_tf to get the same results as the previous implementation
    sublinear_tf=False
)

idf_vectorizer.fit(data["Cleaned Text"])
query_vector = idf_vectorizer.transform([query])

similarity_scores = cosine_similarity(query_vector, doc_vectors)[0]

data["similarity"] = similarity_scores

sorted_data = data.sort_values(by="similarity", ascending=False)

sorted_data.to_csv("similarity_scores.csv", index=False)

print("Data saved to 'similarity_scores.csv'")